In [2]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import playergamelog
import pandas as pd
import time
from IPython. display import clear_output

In [3]:
player_dict = players.get_active_players()

In [4]:
game_logs = []
failures = []
for player in player_dict:
    player_name = player['full_name']
    print(f'Retrieving 2021 game logs for: {player_name}')
    try:
        start = time.time()
        game_log = playergamelog.PlayerGameLog(player_id=player['id'], season='2021').get_data_frames()[0]
        print(f'Completed in {time.time()-start}s\n')
        game_logs.append(game_log)
    except Exception as ex:
        print(f'{type(ex).__name__} FAILED after {time.time() - start}s\n')
        failures.append((player,type(ex).__name__))
    clear_output(wait=True)
        
second_failures = []
print('--- RETRYING FAILURES ---')
for failed in failures:
    player = failed[0]
    player_name = player['full_name']
    print(f'Retrieving 2021 game logs for: {player_name} | Failure reason: {failed[1]}')
    try:
        start = time.time()
        game_log = playergamelog.PlayerGameLog(player_id=player['id'], season='2021').get_data_frames()[0]
        print(f'Completed in {time.time()-start}s\n')
        game_logs.append(game_log)
    except Exception as ex:
        print(f'{type(ex).__name__} FAILED after {time.time() - start}s\n')
        second_failures.append((player,type(ex).__name__))
    clear_output(wait=True)

Retrieving 2021 game logs for: Christian Wood | Failure reason: ConnectionError
Completed in 1.1196699142456055s



In [5]:
master_df = pd.concat(game_logs)
id_name = {player['id']:player['full_name'] for player in player_dict}
master_df['Player_Name'] = master_df['Player_ID'].apply(lambda x: id_name[x])
master_df = master_df[['SEASON_ID',
 'Player_ID',
 'Player_Name',
 'Game_ID',
 'GAME_DATE',
 'MATCHUP',
 'WL',
 'MIN',
 'FGM',
 'FGA',
 'FG_PCT',
 'FG3M',
 'FG3A',
 'FG3_PCT',
 'FTM',
 'FTA',
 'FT_PCT',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 'PLUS_MINUS',
 'VIDEO_AVAILABLE'
]]
master_df.columns = [col.lower for col in master_df.columns]
master_df.to_csv('2021_game_logs.csv',index=False)